In [ ]:
import keras_tuner as kt
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import math
import time
import tensorflow as tf
import psutil
import os

# Function to limit memory usage
def limit_memory(max_memory_gb):
    process = psutil.Process(os.getpid())
    max_memory_bytes = max_memory_gb * 1024 * 1024 * 1024
    process.rlimit(psutil.RLIMIT_AS, (max_memory_bytes, max_memory_bytes))

# Load historical and ground truth data
def load_data(historical_file, ground_truth_file):
    with open(historical_file, 'r') as file:
        historical_data = [list(map(int, line.split())) for line in file]
    
    with open(ground_truth_file, 'r') as file:
        ground_truth_data = [list(map(int, line.split())) for line in file]
    
    return np.array(historical_data), np.array(ground_truth_data)

# Normalize the data
def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Split the data into training and validation sets
def split_data(data, split_ratio=0.95):
    split_index = int(len(data) * split_ratio)
    train_data = data[:split_index]
    val_data = data[split_index:]
    return train_data, val_data

# Prepare the data for LSTM
def prepare_data(data, time_steps=12):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# Build the LSTM model with hyperparameters
def build_model(hp, input_shape):
    model = Sequential()
    units = hp.Int('units', min_value=10, max_value=100, step=10)
    dropout_rate = hp.Float('dropout_rate', min_value=0.05, max_value=0.5, step=0.05)
    learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.001, 0.01, 0.1])
    
    model.add(LSTM(units=units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=12))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Calculate RMSE, MAE, and MAPE
def calculate_metrics(y_true, y_pred):
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape

# Main function to train and evaluate the model
def train_and_evaluate_model(historical_file, ground_truth_file):
    start_time = time.time()  # Start time measurement
    
    # Load and preprocess data
    historical_data, ground_truth_data = load_data(historical_file, ground_truth_file)
    historical_data, scaler = normalize_data(historical_data)
    ground_truth_data = scaler.transform(ground_truth_data)
    
    # Split data into training and validation sets
    train_historical, val_historical = split_data(historical_data)
    train_ground_truth, val_ground_truth = split_data(ground_truth_data)
    
    # Prepare data for LSTM
    X_train, y_train = prepare_data(train_historical)
    X_val, y_val = prepare_data(val_historical)
    
    # Define the input shape
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    # Initialize the tuner
    tuner = kt.RandomSearch(
        lambda hp: build_model(hp, input_shape),
        objective='val_loss',
        max_trials=20,  # Increased number of trials
        executions_per_trial=5,  # Increased number of executions per trial
        directory='my_dir',
        project_name='lstm_tuning'
    )
    
    # Perform hyperparameter tuning
    tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
    
    # Get the best model
    best_model = tuner.get_best_models(num_models=1)[0]
    
    # Predict on validation data
    y_pred = best_model.predict(X_val)
    
    # Inverse transform the predictions and true values
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_val_inv = scaler.inverse_transform(y_val)
    
    # Calculate metrics
    rmse, mae, mape = calculate_metrics(y_val_inv, y_pred_inv)
    
    print(f'Best Parameters: {tuner.get_best_hyperparameters(num_trials=1)[0].values}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape}')
    
    end_time = time.time()  # End time measurement
    print(f'Total Running Time: {end_time - start_time} seconds')

# Usage
train_and_evaluate_model('Train_Historical_Data.json', 'Train_Ground_Truth.json')